In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [5]:
import pandas as pd
from src.inference import fetch_next_hour_predictions, load_batch_of_features_from_store
from src.plot_utils import plot_aggregated_time_series

current_date = pd.Timestamp.now(tz="Etc/UTC")

features = load_batch_of_features_from_store(current_date)

2025-02-12 23:49:37,300 INFO: Initializing external client
2025-02-12 23:49:37,316 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-12 23:49:38,825 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551
Fetching data from 2025-01-15 04:49:37.300344+00:00 to 2025-02-13 03:49:37.300344+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.91s) 
Skipping location_id 109: Not enough data to create even one window.


In [6]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-02-12 05:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,3,2025-02-12 05:00:00
2,0,0,0,0,0,0,0,0,0,0,...,2,3,3,1,1,1,0,0,4,2025-02-12 05:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,2025-02-12 05:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-02-12 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,259,2025-02-12 05:00:00
252,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,260,2025-02-12 05:00:00
253,0,0,0,0,0,0,0,0,0,0,...,12,14,16,3,1,1,0,0,261,2025-02-12 05:00:00
254,0,0,0,0,0,0,0,0,0,0,...,21,10,7,2,0,0,1,7,262,2025-02-12 05:00:00


In [7]:
predictions = fetch_next_hour_predictions()

2025-02-12 23:52:01,897 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-12 23:52:01,908 INFO: Initializing external client
2025-02-12 23:52:01,908 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-12 23:52:03,188 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.37s) 
Current UTC time: 2025-02-13 04:52:01.897542+00:00
Next hour: 2025-02-13 05:00:00+00:00
Querying for date range: 2025-02-13 to 2025-02-14
Filtering for hour: 2025-02-13 05:00
Found 256 records


In [8]:
predictions


,pickup_location_id,predicted_demand,pickup_hour
256,168,1.0,2025-02-13 05:00:00+00:00
257,197,0.0,2025-02-13 05:00:00+00:00
258,169,0.0,2025-02-13 05:00:00+00:00
259,209,11.0,2025-02-13 05:00:00+00:00
260,196,0.0,2025-02-13 05:00:00+00:00
...,...,...,...
507,166,13.0,2025-02-13 05:00:00+00:00
508,17,2.0,2025-02-13 05:00:00+00:00
509,31,0.0,2025-02-13 05:00:00+00:00
510,259,0.0,2025-02-13 05:00:00+00:00


In [16]:
top10 = (
    predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].to_list()
)

In [25]:
top10

[161, 236, 230, 237, 132, 79, 68, 234, 163, 238]

In [17]:
top10

[161, 236, 230, 237, 132, 79, 68, 234, 163, 238]

In [21]:
features["pickup_location_id"] == 22

0      False
1      False
2      False
3      False
4      False
       ...  
251    False
252    False
253    False
254    False
255    False
Name: pickup_location_id, Length: 256, dtype: bool

In [22]:
features_cp = features.copy()

In [44]:
from datetime import timedelta
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]
time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]
time_series_values = [features[col].iloc[0] for col in time_series_columns] + prediction["predicted_demand"].to_list()

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Generate corresponding timestamps for the time series
time_series_dates = pd.date_range(
    start=pickup_hour - timedelta(hours=len(time_series_columns)),
    end=pickup_hour,
    freq="h",
)

# Create a DataFrame for the historical data
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": time_series_values}
)

In [45]:
historical_df

,datetime,rides
0,2025-01-15 05:00:00,0.0
1,2025-01-15 06:00:00,0.0
2,2025-01-15 07:00:00,0.0
3,2025-01-15 08:00:00,0.0
4,2025-01-15 09:00:00,0.0
...,...,...
668,2025-02-12 01:00:00,19.0
669,2025-02-12 02:00:00,4.0
670,2025-02-12 03:00:00,5.0
671,2025-02-12 04:00:00,8.0
